In [1]:
import glob

# Invasive category
invasive_dirs = [
    r'D:\VeligerData\Baylor 2022-03-21_2\Veligers',
    r'D:\VeligerData\invasive',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Zebra Ped 1 To Baylor\Preserved Zebra Ped 1 To Baylor\Sorted Images\Pedi-Zebra Veligers',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Zebra Ped 1a To Baylor\Preserved Zebra Ped 1a To Baylor\Sorted Images\Preserved Zebra Ped 1a',
    r'D:\VeligerData\To Baylor 2023-01-30\To Baylor 2023-01-30\Zebra Pediveliger Image1a\Zebra Pediveligers',
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Baylor Preserved Zebra Umbo 1 Image1\Baylor Preserved Zebra Umbo 1 Image1\Sorted Images\Umbonal',
    ]

# Non-Invasive category
non_invasive_dirs = [
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Baylor Preserved Zebra Umbo 1 Image1\Baylor Preserved Zebra Umbo 1 Image1\Sorted Images\Not',
    r'D:\VeligerData\Baylor 2022-03-21_2\NonVeligers\Images_001',
    r'D:\VeligerData\noninvasive',
    ]

# Ostracod category
ostracod_dirs = [
    r'D:\VeligerData\Ostracod Day 2 Image12 Short To Baylor\Ostracod Day 2 Image12 To Baylor\Sorted Images\Ostracods',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Ostracods 1 To Baylor\Preserved Ostracods 1 To Baylor\Sorted Images\Preserve Ostracods',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Ostracods 1a To Baylor\Preserved Ostracods 1a To Baylor\Sorted Images\Preserved Ostracods 1a',
    r'D:\VeligerData\To Baylor 2023-01-30\To Baylor 2023-01-30\Ostracod Image1\Ostracods1',
    r'D:\VeligerData\Ostracods Day 2 Image1 To Baylor\Ostracods',
    ]

# List to store subdirectories
invasive_subdirs = []
non_invasive_subdirs = []
ostracod_subdirs = []

# Collect subdirectories in the invasive category
for invasive_dir in invasive_dirs:
    invasive_subdirs.extend(glob.glob(invasive_dir))

# Collect subdirectories in the non-invasive category
for non_invasive_dir in non_invasive_dirs:
    non_invasive_subdirs.extend(glob.glob(non_invasive_dir))
    
for ostracod_dir in ostracod_dirs:
    ostracod_subdirs.extend(glob.glob(ostracod_dir))

In [2]:
import glob
from PIL import Image
import os

# List of directories in x
x = non_invasive_subdirs

# List to store subdirectories
y1 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y1.extend(subdirectories)

import glob

# List of directories in x
x = invasive_subdirs

# List to store subdirectories
y2 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y2.extend(subdirectories)

    
import glob

# List of directories in x
x = ostracod_subdirs

# List to store subdirectories
y3 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y3.extend(subdirectories)

    

    
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from tqdm import tqdm


def preprocess_images(y1, label_num, target_size=(28, 28)):
    # List to store images
    images = []
    # List to store labels
    labels = []

    # Process each directory in y1
    for directory in y1:
        # Flag to check if all images in the directory were processed successfully
        all_images_processed = True
        
        for root, dirs, files in os.walk(directory):
            for file in files:
                # Check if the file has an image extension
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    try:
                        # Read the image using PIL
                        image = Image.open(os.path.join(root, file))
                        # Resize the image using tf.image.resize_with_crop_or_pad()
                        image = tf.image.resize_with_crop_or_pad(
                            tf.keras.preprocessing.image.img_to_array(image),
                            target_size[0],
                            target_size[1]
                        )
                        # Normalize the image pixels for ML training
                        image = image / 255.0
                        # Add the preprocessed image to the images list
                        images.append(image)
                    except Exception as e:
                        print(f"Error processing image: {os.path.join(root, file)}. Skipping...")
                        all_images_processed = False
                        break
            
            # If any image in the directory failed to be processed, skip the directory
            if not all_images_processed:
                break

        # If all images in the directory were processed successfully, add the label
        if all_images_processed:
            labels.append(label_num)

    # Convert the images and labels lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    return images, labels




X0,Y0 = preprocess_images(y1, label_num = 0)
X1,Y1 = preprocess_images(y2, label_num = 1)
X2,Y2 = preprocess_images(y3, label_num = 2)

Error processing image: D:\VeligerData\Ostracod Day 2 Image12 Short To Baylor\Ostracod Day 2 Image12 To Baylor\Sorted Images\Ostracods\Object_002\._Image_032.png. Skipping...


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Concatenate the image arrays
train_images = np.concatenate((X0, X1, X2), axis=0)

# Combine the label arrays
train_labels = np.concatenate((Y0, Y1, Y2), axis=0)

# Convert the labels to categorical
train_labels_categorical = to_categorical(train_labels)

# Set the dtype of train_images to float32
train_images = train_images.astype('float32')


# Split the data into train and test sets
from sklearn.utils import shuffle


train_images, train_labels_categorical = shuffle(train_images, train_labels_categorical)

# Now the shuffled data is assigned to the same variable names

X_train, X_test, Y_train, Y_test = train_test_split(train_images, train_labels_categorical, test_size=0.95, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [143968, 4684]

In [ ]:
np.shape(X_train)

In [13]:
DATASET_NAME = "organmnist3d"
BATCH_SIZE = 32
AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = ( 5, 28, 28, 3 )
NUM_CLASSES = 2

# OPTIMIZER
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5

# TRAINING
EPOCHS = 60

# TUBELET EMBEDDING
PATCH_SIZE = (4, 8, 8)
NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE[0]) ** 2

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 32
NUM_HEADS = 2
NUM_LAYERS = 2

#Patch Encoder with Conv2D ,  LSTM , Pos_Emd

class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim, **kwargs):
        super(PatchEncoder, self).__init__(**kwargs)
        self.num_patches = num_patches
        self.projection_dim = projection_dim
        self.projection =keras.Sequential(
            [
                (layers.Conv2D(2, (3, 3), strides=(1,1),activation='relu')),
                TimeDistributed(MaxPooling2D(2,2)),
                TimeDistributed(Flatten()),
                layers.Dense(projection_dim)
            ]) 
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )
        
    def get_config(self):
        config = super().get_config()
        config.update({
            "num_patches": self.num_patches,
            "projection_dim": self.projection_dim,
        })
        return config

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded 
    
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, Flatten, LSTM, Dropout, BatchNormalization
from keras import models

inputs = layers.Input(shape= (5,28,28,3) )

encoded_patches = (PatchEncoder(5, 32 )) (inputs)

for _ in range(6):
    
    x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)

    attention_output= layers.MultiHeadAttention (  num_heads=6, key_dim=32, dropout=0.1 )  (x1, x1)
    
    lstm_output =  LSTM(32,return_sequences=True,dropout=0.1)(x1)

    x2 = layers.Add()([attention_output, encoded_patches,lstm_output])

        # Layer Normalization and MLP
    x3 = layers.LayerNormalization(epsilon=1e-6)(x2)


    x3 = keras.Sequential(layers.Dense(units=32, activation=tf.nn.gelu) )(x3)

        # Skip connection
    encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)

representation = layers.GlobalAvgPool1D()(representation)

#representation = LSTM(100,return_sequences=False,dropout=0.1)(representation)

outputs = layers.Dense(units=3, activation="softmax") ( representation)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

In [ ]:
class_weights = np.zeros(3)

unique_labels, counts = np.unique(train_labels, return_counts=True)

total_samples = np.sum(counts)

for i, label in enumerate(unique_labels):
    class_weights[label] = total_samples / (len(unique_labels) * counts[i])

print("Class weights:", class_weights)

In [14]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=([get_f1]))
class_weights = [1,1,1]
model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1, class_weight = class_weight)

In [5]:
import glob

# Invasive category
invasive_dirs = [
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Ostracods 1a To Baylor\Preserved Ostracods 1a To Baylor\Sorted Images\Not',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Zebra Ped 1 To Baylor\Preserved Zebra Ped 1 To Baylor\Sorted Images\Not',
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Preserved Zebra D-Hinge 1 Baylor\Preserved Zebra D-Hinge 1 Baylor\Sorted Images\Not',
    r'D:\VeligerData\Ostracods Day 2 Image1 To Baylor\Not',
    r'D:\VeligerData\To Baylor 2023-01-30\To Baylor 2023-01-30\Not Veligers\O1',
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Baylor Preserved Zebra Umbo 1 Image1\Baylor Preserved Zebra Umbo 1 Image1\Sorted Images\Not',
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Baylor Preserved Zebra Umbo 1a Image1\Baylor Preserved Zebra Umbo 1a Image1\Sorted Images\Umbonal',
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Preserved Zebra D-Hinge 1 Baylor\Preserved Zebra D-Hinge 1 Baylor\Sorted Images\Zebra D-Hinge',
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Baylor Preserved Zebra Umbo 1 Image1\Baylor Preserved Zebra Umbo 1 Image1\Sorted Images\Umbonal'
]

# Non-Invasive category
non_invasive_dirs = [
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Ostracods 1 To Baylor\Preserved Ostracods 1 To Baylor\Sorted Images\Not',
    r'D:\VeligerData\Ostracods Day 2 Image2 To Baylor\Not',
    r'D:\VeligerData\Ostracods Day 2 Image12 To Baylor_3\Ostracods Day 2 Image12 To Baylor\Sorted Images\Not',
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Baylor Preserved Zebra Umbo 1a Image1\Baylor Preserved Zebra Umbo 1a Image1\Sorted Images\Not'
]

# Ostracod category
ostracod_dirs = [
    r'D:\VeligerData\Ostracod Day 2 Image12 Short To Baylor\Ostracod Day 2 Image12 To Baylor\Sorted Images\Ostracods',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Ostracods 1 To Baylor\Preserved Ostracods 1 To Baylor\Sorted Images\Preserve Ostracods',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Ostracods 1a To Baylor\Preserved Ostracods 1a To Baylor\Sorted Images\Preserved Ostracods 1a',
    r'D:\VeligerData\To Baylor 2023-01-30\To Baylor 2023-01-30\Ostracod Image1\Ostracods1',
    r'D:\VeligerData\Ostracods Day 2 Image1 To Baylor\Ostracods',
    ]

# List to store subdirectories
invasive_subdirs = []
non_invasive_subdirs = []
ostracod_subdirs = []

# Collect subdirectories in the invasive category
for invasive_dir in invasive_dirs:
    invasive_subdirs.extend(glob.glob(invasive_dir))

# Collect subdirectories in the non-invasive category
for non_invasive_dir in non_invasive_dirs:
    non_invasive_subdirs.extend(glob.glob(non_invasive_dir))
    
for ostracod_dir in ostracod_dirs:
    ostracod_subdirs.extend(glob.glob(ostracod_dir))

In [17]:
import glob
from PIL import Image

# List of directories in x
x = non_invasive_subdirs

# List to store subdirectories
y1 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y1.extend(subdirectories)

import glob

# List of directories in x
x = invasive_subdirs

# List to store subdirectories
y2 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y2.extend(subdirectories)

    
import glob

# List of directories in x
x = ostracod_subdirs

# List to store subdirectories
y3 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y3.extend(subdirectories)

    

    
import os
import numpy as np
import tensorflow as tf
from PIL import Image

def preprocess_images(y1, label_num, target_size=(40, 40)):
    # List to store image files
    image_files = []
    # List to store labels
    labels = []

    # Retrieve image files and create labels for each directory in y1
    for directory in y1:
        for root, dirs, files in os.walk(directory):
            for file in files:
                # Check if the file has an image extension
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    # Add the file path to the image_files list
                    image_files.append(os.path.join(root, file))
                    # Add the label to the labels list
                    

    # List to store preprocessed images
    images = []

    # Preprocess each image
    for file in image_files:
        try:
            # Read the image using PIL
            image = Image.open(file)
            # Resize the image using tf.image.resize_with_crop_or_pad()
            image = tf.image.resize_with_crop_or_pad(
                tf.keras.preprocessing.image.img_to_array(image),
                target_size[0],
                target_size[1]
            )
            # Normalize the image pixels for ML training
            image = image / 255.0
            # Add the preprocessed image to the images list
            images.append(image)
            labels.append(label_num)
        except Exception as e:
            print(f"Error processing image: {file}. Skipping...")
            continue

    # Convert the images and labels lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    return images, labels


X0,Y0 = preprocess_images(y1, label_num = 0)
X1,Y1 = preprocess_images(y2, label_num = 1)
X2,Y2 = preprocess_images(y3, label_num = 2)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Concatenate the image arrays
train_images = np.concatenate((X0, X1, X2), axis=0)

# Combine the label arrays
train_labels = np.concatenate((Y0, Y1, Y2), axis=0)

# Convert the labels to categorical
train_labels_categorical = to_categorical(train_labels)

# Set the dtype of train_images to float32
train_images = train_images.astype('float32')


# Split the data into train and test sets
from sklearn.utils import shuffle

# Assuming you have the original train_images and train_labels_categorical

# Shuffle the data
train_images, train_labels_categorical = shuffle(train_images, train_labels_categorical)

# Now the shuffled data is assigned to the same variable names

X_train, X_test1, Y_train, Y_test1 = train_test_split(train_images, train_labels_categorical, test_size=0.99, random_state=42)


In [20]:
class_weights = np.zeros(3)

unique_labels, counts = np.unique(train_labels, return_counts=True)

total_samples = np.sum(counts)

for i, label in enumerate(unique_labels):
    class_weights[label] = total_samples / (len(unique_labels) * counts[i])

print("Class weights:", class_weights)

In [2]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=([get_f1]))
class_weights = [1,1,1]
model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1, class_weight = class_weight)

In [ ]:
pred = model.predict(X_test)
p = np.argmax(pred, axis=1)
res = p
        
test = []
for i in Y_test:
    if(i[0]==1):
        test.append(0)
    elif(i[1]==1):
        test.append(1)
    else:
        test.append(2)
        
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(test,np.argmax(Y_test, axis=1))
print("Accuracy:", accuracy)

In [10]:
# Define the image path
from PIL import Image
image_path = r"D:\VeligerData\Baylor 2022-03-21_2\Veligers\Object_031\Image_009.png"
# Read the image using PIL
image = Image.open(image_path)
# Define the target size for resizing
target_size = (40, 40)
# Resize the image using TensorFlow's resize_with_crop_or_pad function
resized_image = tf.image.resize_with_crop_or_pad( tf.keras.preprocessing.image.img_to_array(image), target_size[0],   target_size[1]
)
# Expand the dimensions to match the model's input shape
expanded_image = tf.expand_dims(resized_image, axis=0)
# Evaluate the TensorFlow tensor and obtain the NumPy array
resized_array = tf.keras.backend.eval(expanded_image)
normalized_image = resized_array / 255.0
x = model.predict(normalized_image)
x

In [ ]:
from tensorflow.keras.models import load_model
# Assuming you have a Keras model named "model"
model.save("five_frame_model.h5")